In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

submissionFile
test_text
test_variants
training_text
training_variants



In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import nltk

train_text = pd.read_csv('../input/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
train_variants = pd.read_csv('../input/training_variants')
test_text = pd.read_csv('../input/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_variants = pd.read_csv('../input/test_variants')

In [3]:
train_df = pd.merge(train_variants,train_text,how='left',on='ID')
test_df = pd.merge(test_variants,test_text,how='left',on='ID')
combine_df = pd.concat([train_df,test_df],ignore_index=True)
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3321 entries, 0 to 3320
Data columns (total 5 columns):
ID           3321 non-null int64
Gene         3321 non-null object
Variation    3321 non-null object
Class        3321 non-null int64
Text         3321 non-null object
dtypes: int64(2), object(3)
memory usage: 155.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5668 entries, 0 to 5667
Data columns (total 4 columns):
ID           5668 non-null int64
Gene         5668 non-null object
Variation    5668 non-null object
Text         5668 non-null object
dtypes: int64(1), object(3)
memory usage: 221.4+ KB


In [ ]:
text_clf = Pipeline([('vect', CountVectorizer(analyzer='word',tokenizer=nltk.word_tokenize,stop_words='english')),
                     ('tfidf',TfidfTransformer()),
                     ('clf',SVC(kernel='linear'))
                    ])
X_all = train_df['Text']
Y_all = train_df['Class']

In [ ]:
X_train, X_cv, Y_train, Y_cv = train_test_split(X_all, Y_all, test_size=0.2)
train = text_clf.fit(X_train,Y_train)
Y_cv_predict = text_clf.predict(X_cv)
np.mean(Y_cv==Y_cv_predict)

0.63909774436090228

In [ ]:
text_clf = text_clf.fit(X_all,Y_all)
Y_test = text_clf.predict(test_df)

In [ ]:
test_df